01_data_loading_and_inspection.ipynb

# Medical Imaging Dataset Inspection

This notebook loads the BreastMNIST dataset and inspects:
- dataset splits,
- image shapes,
- label structure.

## 1. Data loading

We load the BreastMNIST dataset and create separate training, validation, and test splits.
These splits ensure that model development and evaluation remain unbiased.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from medmnist import BreastMNIST
from torchvision import transforms


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = BreastMNIST(split='train', transform=transform, download=True)
val_dataset = BreastMNIST(split='val', transform=transform, download=True)
test_dataset = BreastMNIST(split='test', transform=transform, download=True)

In [ ]:
print("Train samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))
print("Test samples:", len(test_dataset))

## 2. Dataset exploration

We inspect individual samples to understand image dimensions, label format,
and visual characteristics of the data.

In [ ]:
image, label = train_dataset[0]

print("Image shape:", image.shape)
print("Label:", label)

In [ ]:
plt.imshow(image.squeeze(), cmap='gray')
plt.title(f"Label: {label}")
plt.axis('off')
plt.show()

## 3. Class distribution

We analyse the distribution of class labels across training, validation, and test splits
to assess class imbalance and dataset representativeness.

In [ ]:
def get_labels(dataset):
    return dataset.labels.squeeze()

y_train = get_labels(train_dataset)
y_val = get_labels(val_dataset)
y_test = get_labels(test_dataset)

print("Unique labels (train):", np.unique(y_train))

In [ ]:
def class_counts(y):
    labels, counts = np.unique(y, return_counts=True)
    return dict(zip(labels.tolist(), counts.tolist()))

train_counts = class_counts(y_train)
val_counts   = class_counts(y_val)
test_counts  = class_counts(y_test)

print("Train counts:", train_counts)
print("Val counts:", val_counts)
print("Test counts:", test_counts)


In [ ]:
def class_proportions(counts_dict):
    total = sum(counts_dict.values())
    return {k: v / total for k, v in counts_dict.items()}

print("Train proportions:", class_proportions(train_counts))


### Class distribution by split (to assess imbalance and representativeness)

In [ ]:
splits = ["train", "val", "test"]
counts_list = [train_counts, val_counts, test_counts]

all_labels = sorted(set(y_train.tolist()) | set(y_val.tolist()) | set(y_test.tolist()))

x = np.arange(len(all_labels))
width = 0.25

plt.figure(figsize=(8, 4))
for i, (split, counts) in enumerate(zip(splits, counts_list)):
    values = [counts.get(lbl, 0) for lbl in all_labels]
    plt.bar(x + i*width, values, width=width, label=split)

plt.xticks(x + width, all_labels)
plt.xlabel("Class label")
plt.ylabel("Count")
plt.title("Class distribution by split (BreastMNIST)")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import os
os.makedirs("../results", exist_ok=True)

plt.figure(figsize=(8, 4))
for i, (split, counts) in enumerate(zip(splits, counts_list)):
    values = [counts.get(lbl, 0) for lbl in all_labels]
    plt.bar(x + i*width, values, width=width, label=split)

plt.xticks(x + width, all_labels)
plt.xlabel("Class label")
plt.ylabel("Count")
plt.title("Class distribution by split (BreastMNIST)")
plt.legend()
plt.tight_layout()

out_path = "../results/class_distribution_by_split.png"
plt.savefig(out_path, dpi=200)
plt.show()

print("Saved:", out_path)


In [ ]:
train_counts, val_counts, test_counts


In [ ]:
print("Train counts:", train_counts)
print("Validation counts:", val_counts)
print("Test counts:", test_counts)


## 4. Visual characteristics of each class

We visualise example images from each class to assess qualitative differences
and verify that labels correspond to meaningful visual patterns.

### Example images from each class (qualitative inspection)

In [ ]:
def show_examples_for_class(dataset, class_id, n=8):
    idxs = np.where(dataset.labels.squeeze() == class_id)[0][:n]
    plt.figure(figsize=(n*1.2, 2))
    for i, idx in enumerate(idxs):
        img, lab = dataset[idx]
        plt.subplot(1, n, i+1)
        plt.imshow(img.squeeze(), cmap="gray")
        plt.axis("off")
    plt.suptitle(f"Examples for class {class_id} (first {n}")
    plt.tight_layout()
    plt.show()

show_examples_for_class(train_dataset, class_id=0, n=8)
show_examples_for_class(train_dataset, class_id=1, n=8)

### Pixel intensity histograms by class (quantitative comparison)


In [ ]:
def plot_intensity_histogram(dataset, class_id, n=100):
    idxs = np.where(dataset.labels.squeeze() == class_id)[0][:n]
    pixels = []
    for idx in idxs:
        img, _ = dataset[idx]
        pixels.append(img.numpy().ravel())
    pixels = np.concatenate(pixels)

    plt.figure(figsize=(6,4))
    plt.hist(pixels, bins=50)
    plt.title(f"Pixel intensity histogram (class {class_id}, first {n} images)")
    plt.xlabel("Pixel intensity")
    plt.ylabel("Frequency")
    plt.tight_layout()
    out_path = f"../results/pixel_intensity_hist{class_id}.png"
    plt.savefig(out_path, dpi=200)
    plt.show()
    print("Saved:", out_path)

plot_intensity_histogram(train_dataset, class_id=0, n=100)
plot_intensity_histogram(train_dataset, class_id=1, n=100)


## 5. Preprocessing and DataLoaders

We define a preprocessing pipeline based on dataset statistics and construct DataLoaders
to enable efficient and consistent batching during training and evaluation.

### Dataset intensity statistics (mean and standard deviation)

In [ ]:
def compute_mean_std(dataset, n_samples=200):
    pixels = []
    for i in range(min(len(dataset), n_samples)):
        img, _ = dataset[i]
        pixels.append(img.numpy().ravel())
    pixels = np.concatenate(pixels)
    return pixels.mean(), pixels.std()

mean, std = compute_mean_std(train_dataset)
mean, std

In [ ]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[mean], std=[std])
])

### Batch inspection (shape and label verification)

In [ ]:
train_dataset_proc = BreastMNIST(split="train", transform=preprocess, download=False)
val_dataset_proc = BreastMNIST(split="val", transform=preprocess, download=False)
test_dataset_proc = BreastMNIST(split="test", transform=preprocess, download=False)

len(train_dataset_proc), len(val_dataset_proc), len(test_dataset_proc)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset_proc,
    batch_size=32,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset_proc,
    batch_size=32,
    shuffle=False
)

test_loader = DataLoader(
    test_dataset_proc,
    batch_size=32,
    shuffle=False
)

In [ ]:
images, labels = next(iter(train_loader))

images.shape, labels.shape

## 5. Baseline models

We train simple, interpretable baseline models to establish reference performance
before introducing more complex architectures.

In [ ]:
def flatten_dataset(dataset):
    X = []
    y = []
    for img, label in dataset:
        X.append(img.numpy().ravel())
        y.append(int(label.item()))
    return np.array(X), np.array(y)

X_train, y_train = flatten_dataset(train_dataset_proc)
X_val, y_val = flatten_dataset(val_dataset_proc)
X_test, y_test = flatten_dataset(test_dataset_proc)

X_train.shape, y_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)

log_reg.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_val_pred = log_reg.predict(X_val)

print("Validation accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification report (validation):")
print(classification_report(y_val, y_val_pred))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_val, y_val_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues")
plt.title("Confusion matrix - Logistic Regression (Validation)")
out_path = "../results/cm_logreg_test.png"
plt.savefig(out_path, dpi=200)
plt.show()
print("Saved:", out_path)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

y_val_probs = log_reg.predict_proba(X_val)[:, 1]

fpr, tpr, _ = roc_curve(y_val, y_val_probs)
auc = roc_auc_score(y_val, y_val_probs)

plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, label=f"AUC = {auc:.3f}")
plt.plot([0,1], [0,1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Logistic Regression (Validation)")
plt.legend()
plt.tight_layout()

### Logistic Regression - Final Test Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_test_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues")
plt.title("Confusion matrix - Logistic Regression (Test Validation)")
out_path = "../results/cm_logreg_test.png"
plt.savefig(out_path, dpi=200)
plt.show()
print("Saved:", out_path)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

y_test_probs = log_reg.predict_proba(X_test)[:, 1]

fpr, tpr, _ = roc_curve(y_test, y_test_probs)
auc = roc_auc_score(y_test, y_test_probs)

plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, label=f"AUC = {auc:.3f}")
plt.plot([0,1], [0,1], linestyle="--")
plt.xlabel("False Positive")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Logistic Regression (Test Validation)")
plt.legend()
plt.tight_layout()
out_path = "../results/roc_logreg_test.png"
plt.savefig(out_path, dpi=200)
plt.show()
print("Saved:", out_path)


### CNN - Training and Validation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv_block = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(16, 32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(32 * 7 * 7, 64),
        nn.ReLU(),
        nn.Linear(64, 2)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.classifier(x)
        return x

In [ ]:
model = SimpleCNN().to(device)
model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.squeeze().long().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(loader)

In [ ]:
def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.squeeze().long().to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = correct / total
        return running_loss / len(loader), accuracy

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

In [ ]:
best_val_loss = float("inf")
best_state = None

num_epochs = 20

history = []

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)

    history.append((epoch+1, train_loss, val_loss, val_acc))

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

print("Best Val Loss:", best_val_loss)

In [ ]:
model.load_state_dict(best_state)
val_loss, val_acc = evaluate(model, val_loader, criterion)
print("Frozen best model -> Val Loss:", val_loss, "| Val Acc:", val_acc)

### CNN - Final Test Evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score, accuracy_score
import numpy as np

model.eval()

all_labels = []
all_preds = []
all_probs = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.squeeze().long().to(device)

        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)[:, 1]

        preds = torch.argmax(outputs, dim=1)

        all_labels.append(labels.cpu().numpy())
        all_preds.append(preds.cpu().numpy())
        all_probs.append(probs.cpu().numpy())

y_test_true = np.concatenate(all_labels)
y_test_pred = np.concatenate(all_preds)
y_test_prob = np.concatenate(all_probs)

print("Test accuracy:", accuracy_score(y_test_true, y_test_pred))

In [ ]:
print("\nClassification report (test):")
print(classification_report(y_test_true, y_test_pred))

cm = confusion_matrix(y_test_true, y_test_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Confusion matrix - SimpleCNN (Test)")
out_path = "../results/cm_cnn_test.png"
plt.savefig(out_path, dpi=200)
plt.show()
print("Saved:", out_path)

In [ ]:
fpr, tpr, _ = roc_curve(y_test_true, y_test_prob)
auc = roc_auc_score(y_test_true, y_test_prob)

plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, label=f"AUC = {auc:.3f}")
plt.plot([0,1], [0,1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - SimpleCNN (Test)")
plt.legend()
plt.tight_layout()
out_path = "../results/roc_cnn_test.png"
plt.savefig(out_path, dpi=200)
plt.show()
print("Saved:", out_path)

print("Test AUC:", auc)

## 7. Final Summary and Conclusions

### Project Objective

The aim of this project was to evaluate whether machine learning models could reliably classify breast imaging data into two categories. Particular focus was placed on clinically meaningful evaluation, including class imbalance, separability, threshold trade-offs, and generalisation performance.

---

### Baseline Model – Logistic Regression

A logistic regression classifier was implemented using flattened pixel intensities as a linear baseline.

- Validation AUC: 0.860  
- Test AUC: 0.797  

While the model demonstrated meaningful separability, performance decreased on the test set, indicating moderate generalisation gap. Additionally, logistic regression does not capture spatial structure, which is fundamental in medical imaging.

---

### Convolutional Neural Network (CNN)

A small CNN architecture was implemented to model spatial relationships via convolution and pooling.

Early stopping was applied to prevent overfitting and freeze the best validation model.

- Test AUC: 0.862  
- Test accuracy: ~0.82  

Although accuracy remained similar to logistic regression, the CNN significantly improved AUC, indicating stronger intrinsic class separability.

---

### Clinical Interpretation

Improved AUC suggests that the CNN provides better ranking of diseased versus healthy images across thresholds. This allows greater flexibility when selecting operating points, such as prioritising sensitivity in screening contexts.

However, minority class performance remains limited, and false negatives persist. Deployment would require:

- Larger datasets  
- External validation  
- Calibration assessment  
- Clinical risk tolerance analysis  

---

### Limitations

- Small dataset size  
- Class imbalance  
- No external validation cohort  
- Low-resolution (28×28) images compared to real mammography  

These factors limit direct clinical applicability.

---

### Conclusion

This project demonstrates that:

1. Linear models provide a reasonable baseline but are limited by lack of spatial modelling.  
2. Convolutional architectures significantly improve class separability.  
3. AUC provides a more reliable assessment of model capability than accuracy alone.  
4. Clinical evaluation must consider threshold trade-offs and generalisation, not just headline metrics.  

The CNN model showed stronger intrinsic discrimination and would be preferred for further development, pending additional validation.